<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_4_%7C_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from datetime import timedelta
import time
import re

# --- CONFIGURACIÓN ---
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)
geolocator = Nominatim(user_agent="tracker_debug_v12")

def obtener_direccion(lat, lon):
    try:
        # time.sleep(0.5) # Reducimos tiempo para pruebas
        location = geolocator.reverse((lat, lon), exactly_one=True)
        return location.address.split(',')[0] if location else "Ubicación desconocida"
    except:
        return "Mapa no disponible"

def encontrar_archivo():
    rutas = ['/content/', '/content/sample_data/']
    for r in rutas:
        if os.path.exists(r):
            for f in os.listdir(r):
                if (f.endswith('.xls') or f.endswith('.xlsx')) and "california" not in f:
                    return os.path.join(r, f)
    return None

def limpiar_coordenada(coord_str):
    # Función inteligente para arreglar formatos 4,123 o 4.123
    s = str(coord_str).strip()
    # Si viene vacía
    if not s or s.lower() == 'nan': return None

    try:
        # Caso 1: Formato estándar "4.123, -73.123"
        if ',' in s and '.' in s:
            parts = s.split(',')
            if len(parts) == 2: return float(parts[0]), float(parts[1])

        # Caso 2: Formato español posible "4,123 , -73,123" (muchas comas)
        # Intentamos reemplazar comas decimales por puntos si hay espacios
        s = s.replace(" ", "")
        # Si hay una coma separando lat y lon, suele dividir en dos numeros grandes
        parts = s.split(',')
        if len(parts) == 2:
            return float(parts[0]), float(parts[1])
        elif len(parts) > 2:
            # Caso dificil: 4,123,-73,123 -> Asumimos que la coma del medio separa
            # Esto es arriesgado, mejor intentamos limpiar chars raros
            pass

        # Intento bruto: Split por coma, convertir a float
        lat, lon = map(float, s.split(','))
        return lat, lon
    except:
        return None

def generar_reporte_debug():
    archivo = encontrar_archivo()
    if not archivo: return "❌ No encuentro el archivo."

    print(f"📂 Procesando: {archivo}")
    try:
        df = pd.read_excel(archivo)
    except:
        df = pd.read_csv(archivo) # Fallback

    df.columns = df.columns.str.upper().str.strip()
    print(f"Columnas: {list(df.columns)}") # DEBUG

    if 'FHEVENTO' not in df.columns: return "❌ Falta columna FHEVENTO"

    df['FHEVENTO'] = pd.to_datetime(df['FHEVENTO'])
    df = df.sort_values(by='FHEVENTO').reset_index(drop=True)

    reporte = []

    # Estados
    ya_reporto_encendido = False
    estacionado_activo = False
    est_inicio = None
    est_fin = None
    est_coords = None
    dentro_bodega = False

    print("\n--- INICIANDO ANÁLISIS FILA POR FILA (Primeras 10 filas de muestra) ---")

    eventos_detectados = 0

    for i, row in df.iterrows():
        raw_evento = str(row['EVENTO']).upper().strip()
        fecha = row['FHEVENTO']
        hora_fmt = fecha.strftime('%I:%M %p').lower()

        # 1. INTENTO DE LEER COORDENADAS (SIN try-except silencioso)
        coords_raw = row['COORDS']
        res = limpiar_coordenada(coords_raw)

        if res is None:
            if i < 5: print(f"⚠️ Fila {i}: Error leyendo coordenadas '{coords_raw}'. Saltando.")
            continue

        lat, lon = res
        punto = Point(lat, lon)
        coords_tuple = (lat, lon)

        # DEBUG VISUAL (Solo al principio para no saturar)
        if i < 5: print(f"✅ Fila {i}: {raw_evento} | Coords OK: {lat}, {lon}")

        # --- LOGICA BODEGA ---
        en_bodega_ahora = bodega_polygon.contains(punto)
        if en_bodega_ahora and not dentro_bodega:
            msg = f"-{hora_fmt} Entrada Bodega Monte Carlo"
            reporte.append(msg)
            print(f"  -> AGREGADO: {msg}")
            dentro_bodega = True
        elif not en_bodega_ahora and dentro_bodega:
            msg = f"-{hora_fmt} Salida Bodega Monte Carlo"
            reporte.append(msg)
            print(f"  -> AGREGADO: {msg}")
            dentro_bodega = False

        # --- LOGICA ENCENDIDO ---
        if "ENCENDIDO" in raw_evento and not ya_reporto_encendido:
            lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(lat, lon)
            msg = f"-{hora_fmt} Se enciende en {lug}"
            reporte.append(msg)
            print(f"  -> AGREGADO: {msg}")
            ya_reporto_encendido = True
            eventos_detectados += 1

        # --- LOGICA ESTACIONADO ---
        es_quieto = False
        if "ESTACIONADO" in raw_evento:
            es_quieto = True
        elif estacionado_activo:
            dist = geodesic(est_coords, coords_tuple).meters
            if dist < 60: es_quieto = True

        if es_quieto:
            if not estacionado_activo:
                estacionado_activo = True
                est_inicio = fecha
                est_coords = coords_tuple
                print(f"  -> (Inicia ciclo estacionado interno a las {hora_fmt})")
            est_fin = fecha
        else:
            if estacionado_activo:
                mins = (est_fin - est_inicio).total_seconds() / 60
                print(f"  -> (Cierra ciclo estacionado. Duración: {mins} min)")
                if mins > 2:
                    lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(*est_coords)
                    h_ini = est_inicio.strftime('%I:%M')
                    h_fin = est_fin.strftime('%I:%M %p').lower()
                    msg = f"-{h_ini} / {h_fin} Vehículo estacionado en {lug}"
                    reporte.append(msg)
                    print(f"  -> AGREGADO REPORTE: {msg}")
                    eventos_detectados += 1
                estacionado_activo = False
                est_inicio = None
                est_fin = None

        # --- LOGICA CIUDAD ---
        if ("ENTRADA" in raw_evento or "SALIDA" in raw_evento) and "BODEGA" not in raw_evento:
            msg = f"-{hora_fmt} {raw_evento.title()}"
            reporte.append(msg)
            print(f"  -> AGREGADO: {msg}")
            eventos_detectados += 1

    # Cierre final
    if estacionado_activo:
        lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(*est_coords)
        h_ini = est_inicio.strftime('%I:%M')
        h_fin = est_fin.strftime('%I:%M %p').lower()
        msg = f"-{h_ini} / {h_fin} Vehículo estacionado en {lug}"
        reporte.append(msg)
        print(f"  -> AGREGADO FINAL: {msg}")

    print(f"\n✨ Proceso terminado. Total eventos en reporte: {len(reporte)}")
    return ", ".join(reporte) + "."

resultado = generar_reporte_debug()
print("\n--- REPORTE FINAL ---")
print(resultado)

📂 Procesando: /content/sample_data/reporte-NCX43H-15-01-2026-10-06-12.xlsx
Columnas: ['EVENTO', 'FHEVENTO', 'VEL', 'COORDS', 'ODOM', 'UNNAMED: 5']

--- INICIANDO ANÁLISIS FILA POR FILA (Primeras 10 filas de muestra) ---
⚠️ Fila 0: Error leyendo coordenadas '4.1344433 -73.640965'. Saltando.
⚠️ Fila 1: Error leyendo coordenadas '4.1344433 -73.640965'. Saltando.
⚠️ Fila 2: Error leyendo coordenadas '4.1344433 -73.640965'. Saltando.
⚠️ Fila 3: Error leyendo coordenadas '4.1344433 -73.640965'. Saltando.
⚠️ Fila 4: Error leyendo coordenadas '4.1344433 -73.640965'. Saltando.

✨ Proceso terminado. Total eventos en reporte: 0

--- REPORTE FINAL ---
.
